# New federated Aggregation with `fluke`

This tutorial will guide you through the steps required to implement KRUM and Multi-KRUM with ``fluke``.

```{attention}
This tutorial does not go into the details of the implementation, but it provides a quick overview of the steps required to implement a new federated learning algorithm.
```

Try this notebook: [![Open in Colab](https://img.shields.io/badge/Open_in_Colab-blue?style=flat-square&logo=google-colab&logoColor=yellow&labelColor=gray)
](https://colab.research.google.com/github/CasellaJr/Fluke-tutorial-ECAI25/blob/main/2_fluke_krum.ipynb)

## Install `fluke` (if not already done)

In [11]:
!pip install fluke-fl

# KRUM

Let's do a further operation, let's implement KRUM aggregation, that should be more robust to byzantine attacks than the median aggregation.

In [12]:
from typing import Collection, Sequence, Iterable
from torch.nn import Module
import torch
from fluke.client import Client
from fluke.server import Server
import numpy as np
from copy import deepcopy
from fluke.data import FastDataLoader


class MyServer(Server):
    def __init__(
        self,
        model: torch.nn.Module,
        test_set: FastDataLoader | None,
        clients: Sequence[Client],
        weighted: bool = False,
        lr: float = 1.0,
        f: int = 0,
        **kwargs,
    ):
        super().__init__(model, test_set, clients, weighted, lr, **kwargs)
        self.hyper_params.update(f=f)


    def aggregate(self, eligible: Sequence[Client], client_models: Iterable[Module]) -> None:
            
            client_models = list(client_models)
            n = len(client_models)
            f = self.hyper_params.f
            
            server_param_items = list(self.model.named_parameters())
            param_keys = [k for k, _ in server_param_items]

            # get one vector for each client
            client_vecs = []
            for i, cm in enumerate(client_models):
                cm_state = dict(cm.named_parameters()) 
                parts = [torch.ravel(cm_state[k].data) for k in param_keys]
                vec = torch.cat(parts, dim=0) 
                client_vecs.append(vec)

            # put all these vector in a tensor
            mat = torch.stack(client_vecs, dim=0)
            
            # calculate pairwise euclidean distance matrix to extract the scores
            dists = torch.cdist(mat, mat, p=2.0)  # shape (n, n)
            sq_dists = dists.pow(2)

            scores = torch.empty(n)
            # For each model i, sort distances to others (exclude self-distance=0)
            # and sum the smallest (n - f - 2) distances
            nb_small = n - f - 2
            for i in range(n):
                # distances to others (includes zero at i)
                row = sq_dists[i]
                # Get sorted distances (ascending)
                sorted_row, _ = torch.sort(row)
                selected = sorted_row[1: nb_small + 1]
                scores[i] = selected.sum()

            _, selected_indices = torch.topk(scores, k=1, largest=False) 

            selected_indices = selected_indices.tolist()  # indices of chosen clients

            chosen_idx = selected_indices[0]
            chosen_state = dict(client_models[chosen_idx].named_parameters())
            # Copy parameter tensors into server model
            for k in param_keys:
                self.model.state_dict()[k].copy_(chosen_state[k].data)

            return deepcopy(self.model)

In [13]:
class MyClient(Client):

    # we override the fit method to implement our training "strategy"
    def fit(self, override_local_epochs: int = 0) -> float:
        # we can override the number of local epochs and call the parent class method
        new_local_epochs = np.random.randint(1, self.hyper_params.local_epochs + 1)
        return super().fit(new_local_epochs)

## Implementing the new federated algorithm

Now, we only need to put everything together in a new class that inherits from `fluke.algorithms.CentralizedFL` specifying the server and client classes we just implemented.

In [4]:
from fluke.algorithms import CentralizedFL

class MyFLAlgorithm(CentralizedFL):

    def get_client_class(self) -> type[Client]:
        return MyClient

    def get_server_class(self) -> type[Server]:
        return MyServer

Everything is ready! Now we can test our new federated algorithm with `fluke`!

## Ready to test KRUM

In [14]:
from fluke.data import DataSplitter
from fluke.data.datasets import Datasets
from fluke import DDict
from fluke.utils.log import Log
from fluke.evaluation import ClassificationEval
from fluke import FlukeENV

env = FlukeENV()
env.set_seed(42) # we set a seed for reproducibility
env.set_device("cpu") # we use the CPU for this example

dataset = Datasets.get("mnist", path="./data")

# we set the evaluator to be used by both the server and the clients
env.set_evaluator(ClassificationEval(eval_every=1, n_classes=dataset.num_classes))

splitter = DataSplitter(dataset=dataset,
                        distribution="iid")

client_hp = DDict(
    batch_size=10,
    local_epochs=5,
    loss="CrossEntropyLoss",
    optimizer=DDict(
      lr=0.01,
      momentum=0.9,
      weight_decay=0.0001),
    scheduler=DDict(
      gamma=1,
      step_size=1)
)

# we put together the hyperparameters for the algorithm
hyperparams = DDict(client=client_hp,
                    server=DDict(weighted=True, f=7),
                    model="MNIST_2NN")

Here is where the new federated algorithm comes into play.

In [15]:
algorithm = MyFLAlgorithm(n_clients=10, # 10 clients in the federation
                          data_splitter=splitter,
                          hyper_params=hyperparams)

logger = Log()
algorithm.set_callbacks(logger)
logger.init()
algorithm.run(n_rounds=3, eligible_perc=1)

╭──────────────────────────────────────────── Round: 1 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.8801,                                                                      │
│         'macro_precision': 0.88894,                                                              │
│         'macro_recall': 0.87951,                                                                 │
│         'macro_f1': 0.8788,                                                                      │
│         'micro_precision': 0.8801,                                                               │
│         'micro_recall': 0.8801,                                                                  │
│         'micro_f1': 0.8801,                                                                      │
│         'round': 1                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 281.1 MB [3.66 %]

╭──────────────────────────────────────────── Round: 2 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9395,                                                                      │
│         'macro_precision': 0.9402,                                                               │
│         'macro_recall': 0.93883,                                                                 │
│         'macro_f1': 0.93883,                                                                     │
│         'micro_precision': 0.9395,                                                               │
│         'micro_recall': 0.9395,                                                                  │
│         'micro_f1': 0.9395,                                                                      │
│         'round': 2                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 382.9 MB [3.50 %]

╭──────────────────────────────────────────── Round: 3 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9435,                                                                      │
│         'macro_precision': 0.9454,                                                               │
│         'macro_recall': 0.94257,                                                                 │
│         'macro_f1': 0.94295,                                                                     │
│         'micro_precision': 0.9435,                                                               │
│         'micro_recall': 0.9435,                                                                  │
│         'micro_f1': 0.9435,                                                                      │
│         'round': 3                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 357.4 MB [3.53 %]

Output()

╭────────────────────────────────────── Overall Performance ───────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9435,                                                                      │
│         'macro_precision': 0.9454,                                                               │
│         'macro_recall': 0.94257,                                                                 │
│         'macro_f1': 0.94295,                                                                     │
│         'micro_precision': 0.9435,                                                               │
│         'micro_recall': 0.9435,                                                                  │
│         'micro_f1': 0.9435,                                                                      │
│         'round': 3                                                                               │
│     },                                                                                           │
│     'comm_costs': 12467700                                                                       │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

We only just need to run it!

# Multi-KRUM

In [16]:
class MyServer(Server):
    def __init__(
        self,
        model: torch.nn.Module,
        test_set: FastDataLoader | None,
        clients: Sequence[Client],
        weighted: bool = False,
        lr: float = 1.0,
        f: int = 0,
        m: int = 1,
        **kwargs,
    ):
        super().__init__(model, test_set, clients, weighted, lr, **kwargs)
        self.hyper_params.update(f=f, m=m)


    def aggregate(self, eligible: Sequence[Client], client_models: Iterable[Module]) -> None:
            
            client_models = list(client_models)
            n = len(client_models)
            f = self.hyper_params.f
            
            server_param_items = list(self.model.named_parameters())
            param_keys = [k for k, _ in server_param_items]

            # get one vector for each client
            client_vecs = []
            for i, cm in enumerate(client_models):
                cm_state = dict(cm.named_parameters()) 
                parts = [torch.ravel(cm_state[k].data) for k in param_keys]
                vec = torch.cat(parts, dim=0) 
                client_vecs.append(vec)

            # put all these vector in a tensor
            mat = torch.stack(client_vecs, dim=0)
            
            # calculate pairwise euclidean distance matrix to extract the scores
            dists = torch.cdist(mat, mat, p=2.0)  # shape (n, n)
            sq_dists = dists.pow(2)

            scores = torch.empty(n)
            # For each model i, sort distances to others (exclude self-distance=0)
            # and sum the smallest (n - f - 2) distances
            nb_small = n - f - 2
            for i in range(n):
                # distances to others (includes zero at i)
                row = sq_dists[i]
                # Get sorted distances (ascending)
                sorted_row, _ = torch.sort(row)
                selected = sorted_row[1: nb_small + 1]
                scores[i] = selected.sum()

            _, selected_indices = torch.topk(scores, k=self.hyper_params.m, largest=False)

            selected_indices = selected_indices.tolist()  # indices of chosen clients

            # If m == 1: KRUM
            if self.hyper_params.m == 1:
                chosen_idx = selected_indices[0]
                chosen_state = dict(client_models[chosen_idx].named_parameters())
                # Copy parameter tensors into server model
                for k in param_keys:
                    self.model.state_dict()[k].copy_(chosen_state[k].data)
            else:
                # Multi-KRUM: average the selected m models elementwise (simple mean)
                # We'll build averaged parameters by stacking the selected params and mean them
                # Using named_parameters order to assemble results
                # Initialize accumulator dict of tensors (float) on server device
                accum = {}
                for k in param_keys:
                    first_t = client_models[selected_indices[0]].state_dict()[k].data
                    accum[k] = first_t.clone().detach().float()  # use float accumulation

                for idx in selected_indices[1:]:
                    cm_state = client_models[idx].state_dict()
                    for k in param_keys:
                        accum[k].add_(cm_state[k].data)

                # divide by m and copy back
                for k in param_keys:
                    mean_t = (accum[k] / float(self.hyper_params.m)).to(self.model.state_dict()[k].dtype)
                    self.model.state_dict()[k].copy_(mean_t)

            return deepcopy(self.model)

In [8]:
from fluke.algorithms import CentralizedFL

class MyFLAlgorithm(CentralizedFL):

    def get_client_class(self) -> type[Client]:
        return MyClient

    def get_server_class(self) -> type[Server]:
        return MyServer

In [9]:
from fluke.data import DataSplitter
from fluke.data.datasets import Datasets
from fluke import DDict
from fluke.utils.log import Log
from fluke.evaluation import ClassificationEval
from fluke import FlukeENV

env = FlukeENV()
env.set_seed(42) # we set a seed for reproducibility
env.set_device("cpu") # we use the CPU for this example

dataset = Datasets.get("mnist", path="./data")

# we set the evaluator to be used by both the server and the clients
env.set_evaluator(ClassificationEval(eval_every=1, n_classes=dataset.num_classes))

splitter = DataSplitter(dataset=dataset,
                        distribution="iid")

client_hp = DDict(
    batch_size=10,
    local_epochs=5,
    loss="CrossEntropyLoss",
    optimizer=DDict(
      lr=0.01,
      momentum=0.9,
      weight_decay=0.0001),
    scheduler=DDict(
      gamma=1,
      step_size=1)
)

# we put together the hyperparameters for the algorithm
hyperparams = DDict(client=client_hp,
                    server=DDict(weighted=True, f=5, m=2),
                    model="MNIST_2NN")

In [10]:
algorithm = MyFLAlgorithm(n_clients=10, # 10 clients in the federation
                          data_splitter=splitter,
                          hyper_params=hyperparams)

logger = Log()
algorithm.set_callbacks(logger)
logger.init()
algorithm.run(n_rounds=3, eligible_perc=1)

╭──────────────────────────────────────────── Round: 1 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9081,                                                                      │
│         'macro_precision': 0.90874,                                                              │
│         'macro_recall': 0.90739,                                                                 │
│         'macro_f1': 0.90673,                                                                     │
│         'micro_precision': 0.9081,                                                               │
│         'micro_recall': 0.9081,                                                                  │
│         'micro_f1': 0.9081,                                                                      │
│         'round': 1                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 487.9 MB [6.91 %]

╭──────────────────────────────────────────── Round: 2 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9422,                                                                      │
│         'macro_precision': 0.94195,                                                              │
│         'macro_recall': 0.94156,                                                                 │
│         'macro_f1': 0.94148,                                                                     │
│         'micro_precision': 0.9422,                                                               │
│         'micro_recall': 0.9422,                                                                  │
│         'micro_f1': 0.9422,                                                                      │
│         'round': 2                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 339.4 MB [3.36 %]

╭──────────────────────────────────────────── Round: 3 ────────────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9505,                                                                      │
│         'macro_precision': 0.95035,                                                              │
│         'macro_recall': 0.95015,                                                                 │
│         'macro_f1': 0.9499,                                                                      │
│         'micro_precision': 0.9505,                                                               │
│         'micro_recall': 0.9505,                                                                  │
│         'micro_f1': 0.9505,                                                                      │
│         'round': 3                                                                               │
│     },                                                                                           │
│     'comm_cost': 3562200                                                                         │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Memory usage: 271.2 MB [3.70 %]

Output()

╭────────────────────────────────────── Overall Performance ───────────────────────────────────────╮
│ {                                                                                                │
│     'global': {                                                                                  │
│         'accuracy': 0.9505,                                                                      │
│         'macro_precision': 0.95035,                                                              │
│         'macro_recall': 0.95015,                                                                 │
│         'macro_f1': 0.9499,                                                                      │
│         'micro_precision': 0.9505,                                                               │
│         'micro_recall': 0.9505,                                                                  │
│         'micro_f1': 0.9505,                                                                      │
│         'round': 3                                                                               │
│     },                                                                                           │
│     'comm_costs': 12467700                                                                       │
│ }                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯